# 无layer和block这种层级关系
直接调用torchvision里写好的resnet会参杂很多不必要元素，这对我后续调整网络结构是一个很大的阻碍，这份代码可以让我非常清晰明了的知道ResNet-18的各层细节，没有各种函数调用绕来绕去，当然因为是针对目标检测模型设计的，所以我去掉了最后两层（AvgPool+FC），因为命名方式和层级关系（layer和block）跟PyTorch官方的代码不一样，所以不能使用官方提供的Pre-Training参数

In [1]:
import torch
import torch.nn as nn

class Backbone(nn.Module):
    def __init__(self):
        super(Backbone, self).__init__()

        # ResNet's Head
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu1 = nn.ReLU(inplace=True)
        self.maxpool1 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # ResNet Block-1 
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(64)
        self.relu2 = nn.ReLU(inplace=True)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn3 = nn.BatchNorm2d(64)
        self.relu3 = nn.ReLU(inplace=True)

        self.conv4 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn4 = nn.BatchNorm2d(64)
        self.relu4 = nn.ReLU(inplace=True)
        self.conv5 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn5 = nn.BatchNorm2d(64)
        self.relu5 = nn.ReLU(inplace=True)

        # ResNet Block-2 
        self.conv6 = nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1, bias=False)
        self.bn6 = nn.BatchNorm2d(128)
        self.relu6 = nn.ReLU(inplace=True)
        self.conv7 = nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn7 = nn.BatchNorm2d(128)
        self.relu7 = nn.ReLU(inplace=True)
        self.downsample1 = nn.Sequential(nn.Conv2d(64, 128, kernel_size=1, stride=2, bias=False),
                                         nn.BatchNorm2d(128))
        
        self.conv8 = nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn8 = nn.BatchNorm2d(128)
        self.relu8 = nn.ReLU(inplace=True)
        self.conv9 = nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn9 = nn.BatchNorm2d(128)
        self.relu9 = nn.ReLU(inplace=True)

        # ResNet Block-3
        self.conv10 = nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1, bias=False)
        self.bn10 = nn.BatchNorm2d(256)
        self.relu10 = nn.ReLU(inplace=True)
        self.conv11 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn11 = nn.BatchNorm2d(256)
        self.relu11 = nn.ReLU(inplace=True)
        self.downsample2 = nn.Sequential(nn.Conv2d(128, 256, kernel_size=1, stride=2, bias=False),
                                         nn.BatchNorm2d(256))
        
        self.conv12 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn12 = nn.BatchNorm2d(256)
        self.relu12 = nn.ReLU(inplace=True)
        self.conv13 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn13 = nn.BatchNorm2d(256)
        self.relu13 = nn.ReLU(inplace=True)

        # ResNet Block-4
        self.conv14 = nn.Conv2d(256, 512, kernel_size=3, stride=2, padding=1, bias=False)
        self.bn14 = nn.BatchNorm2d(512)
        self.relu14 = nn.ReLU(inplace=True)
        self.conv15 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn15 = nn.BatchNorm2d(512)
        self.relu15 = nn.ReLU(inplace=True)
        self.downsample3 = nn.Sequential(nn.Conv2d(256, 512, kernel_size=1, stride=2, bias=False),
                                         nn.BatchNorm2d(512))
        
        self.conv16 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn16 = nn.BatchNorm2d(512)
        self.relu16 = nn.ReLU(inplace=True)
        self.conv17 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn17 = nn.BatchNorm2d(512)
        self.relu17 = nn.ReLU(inplace=True)


    def forward(self, x):
        C1 = self.conv1(x)
        C1 = self.bn1(C1)
        C1 = self.relu1(C1)
        C1 = self.maxpool1(C1)

        # ResNet Block-1
        identity = C1
        C2 = self.conv2(C1)
        C2 = self.bn2(C2)
        C2 = self.relu2(C2)
        C3 = self.conv3(C2)
        C3 = self.bn3(C3)
        C3 = self.relu3(C3 + identity)

        identity = C3
        C4 = self.conv2(C3)
        C4 = self.bn2(C4)
        C4 = self.relu2(C4)
        C5 = self.conv3(C4)
        C5 = self.bn3(C5)
        C5 = self.relu3(C5 + identity)

        # ResNet Block-2
        identity = C5
        C6 = self.conv2(C5)
        C6 = self.bn2(C6)
        C6 = self.relu2(C6)
        C7 = self.conv3(C6)
        C7 = self.bn3(C7)
        C7 = self.relu3(C7 + self.downsample1(identity))

        identity = C7
        C8 = self.conv2(C7)
        C8 = self.bn2(C8)
        C8 = self.relu2(C8)
        C9 = self.conv3(C8)
        C9 = self.bn3(C9)
        C9 = self.relu3(C9 + identity)

        # ResNet Block-3
        identity = C9
        C10 = self.conv2(C9)
        C10 = self.bn2(C10)
        C10 = self.relu2(C10)
        C11 = self.conv3(C10)
        C11 = self.bn3(C11)
        C11 = self.relu3(C11 + self.downsample2(identity))

        identity = C11
        C12 = self.conv2(C11)
        C12 = self.bn2(C12)
        C12 = self.relu2(C12)
        C13 = self.conv3(C12)
        C13 = self.bn3(C13)
        C13 = self.relu3(C13 + identity)

        # ResNet Block-4
        identity = C13
        C14 = self.conv2(C3)
        C14 = self.bn2(C14)
        C14 = self.relu2(C14)
        C15 = self.conv3(C14)
        C15 = self.bn3(C15)
        C15 = self.relu3(C15 + self.downsample2(identity))

        identity = C15
        C16 = self.conv2(C15)
        C16 = self.bn2(C16)
        C16 = self.relu2(C16)
        C17 = self.conv3(C16)
        C17 = self.bn3(C17)
        C17 = self.relu3(C17 + identity)



        

model = Backbone()
#model.load_state_dict(torch.load('resnet18.pth'), strict=False)
# print(model) # 打印模型的网络结构
pre = torch.load('resnet18.pth') # 下载地址：https://download.pytorch.org/models/resnet18-5c106cde.pth
pre = [k for k, v in pre.items()]
# print(pre)
#model.state_dict().keys()


# 有layer和block这种层级关系
为了能用上PyTorch提供的Pre-Training参数，所以还是得设计出跟官方一样的层级关系和命名方式，值得注意的是，这并不会影响你对网络结构进行调整。当然，前提是层级关系和命名保持不变，具体可以参考这篇文章：[pytorch修改网络结构后如何加装预训练模型](https://blog.csdn.net/qq_36414085/article/details/112652317)


In [7]:
import torch
import torch.nn as nn

class BasicBlock(nn.Module):

    def __init__(self, conv1_in, conv1_out, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        
        self.conv1 = nn.Conv2d(conv1_in, conv1_out, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(conv1_out)
        self.relu = nn.ReLU(inplace=True)
        
        self.conv2 = nn.Conv2d(conv1_out, conv1_out, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(conv1_out)
        
        # downsample是增加x通道数用的，因为下面一层的F(x)可能会增加通道数，前面一层无法直接相加
        self.downsample = downsample

    def forward(self, x):
        identity = x # 上一层的信息

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out) # conv-1有ReLU

        out = self.conv2(out)
        out = self.bn2(out) # 注意到这里conv-2没有ReLU

        if self.downsample is not None:
            identity = self.downsample(x) # 没记错的李沐课里说的是用1x1卷积来增加通道数

        out += identity
        out = self.relu(out) # 注意到，联合上一层信息之后在进行ReLU
        return out


class Backbone(nn.Module):
    def __init__(self):
        super(Backbone, self).__init__()

        # ResNet's Head
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # ResNet Block-1
        self.layer1 = nn.Sequential(BasicBlock(64, 64), 
                                    BasicBlock(64, 64))

        # ResNet Block-2 
        downsample = nn.Sequential(nn.Conv2d(64, 128, kernel_size=1, stride=2, bias=False),
                                   nn.BatchNorm2d(128))

        self.layer2 = nn.Sequential(BasicBlock(64, 128, 2, downsample=downsample), 
                                    BasicBlock(128, 128))

        # ResNet Block-3
        downsample = nn.Sequential(nn.Conv2d(128, 256, kernel_size=1, stride=2, bias=False),
                                   nn.BatchNorm2d(256))
        self.layer3 = nn.Sequential(BasicBlock(128, 256, 2, downsample=downsample), 
                                    BasicBlock(256, 256))

        # ResNet Block-4
        downsample = nn.Sequential(nn.Conv2d(256, 512, kernel_size=1, stride=2, bias=False),
                                   nn.BatchNorm2d(512))
        self.layer4 = nn.Sequential(BasicBlock(256, 512, 2, downsample=downsample), 
                                    BasicBlock(512, 512))
        

    def forward(self, x):
        C_1 = self.conv1(x)
        C_1 = self.bn1(C_1)
        C_1 = self.relu(C_1)
        C_1 = self.maxpool(C_1)

        C_2 = self.layer1(C_1)
        C_3 = self.layer2(C_2)
        C_4 = self.layer3(C_3)
        C_5 = self.layer4(C_4)

        return C_5



model = Backbone()
#model.load_state_dict(torch.load('resnet18.pth'), strict=False)
# print(model) # 打印模型的网络结构
pre = torch.load('resnet18.pth') # 下载地址：https://download.pytorch.org/models/resnet18-5c106cde.pth
pre = [k for k, v in pre.items()]
# print(pre)
model.state_dict().keys()
model.load_state_dict(torch.load('resnet18.pth'), strict=False)

_IncompatibleKeys(missing_keys=['conv1.bias'], unexpected_keys=['fc.weight', 'fc.bias'])